# Системы хранения и обработки данных. Домашнее задание 3.
## Присяжнюк Артем Сергеевич

# Подготовка

In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2 as pg

In [2]:
conn = pg.connect(dbname="postgres", host="localhost", user="postgres", password="password", port=5432)
cursor = conn.cursor()

# Задание 1

## Создание таблиц

In [198]:
cursor.execute("""
               create table transaction (
                   transaction_id int4 NOT NULL PRIMARY KEY
                   ,product_id int4
                   ,customer_id int4 
                   ,transaction_date varchar
                   ,online_order varchar
                   ,order_status varchar
                   ,list_price float4
                   ,brand varchar
                   ,product_line varchar
                   ,product_class varchar
                   ,product_size varchar
                   ,standard_cost float4                 
               )
               """)

In [199]:
cursor.execute("""
               create table customer (
                   customer_id int4 NOT NULL PRIMARY KEY
                   ,first_name varchar
                   ,last_name varchar
                   ,gender varchar
                   ,dob varchar
                   ,job_title varchar
                   ,job_industry_category varchar
                   ,wealth_segment varchar
                   ,deceased_indicator varchar
                   ,owns_car varchar
                   ,property_valuation int4
                   ,address varchar
                   ,postcode varchar
                   ,state varchar
                   ,country varchar                  
               )
               """)

In [200]:
conn.commit()

## Обработка данных

In [86]:
import pandas as pd

df_transaction = pd.read_csv("data/transaction.csv", sep=';')
df_transaction.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"
2,3,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,"1793,43","248,82"
3,4,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,"1198,46","381,10"
4,5,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,"1765,3","709,48"


In [202]:
df_transaction.standard_cost = df_transaction.standard_cost.apply(lambda x: float(str(x).replace(',', '.')))
df_transaction.list_price = df_transaction.list_price.apply(lambda x: float(str(x).replace(',', '.')))

In [203]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    20000 non-null  int64  
 1   product_id        20000 non-null  int64  
 2   customer_id       20000 non-null  int64  
 3   transaction_date  20000 non-null  object 
 4   online_order      19640 non-null  object 
 5   order_status      20000 non-null  object 
 6   brand             19803 non-null  object 
 7   product_line      19803 non-null  object 
 8   product_class     19803 non-null  object 
 9   product_size      19803 non-null  object 
 10  list_price        20000 non-null  float64
 11  standard_cost     19803 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 1.8+ MB


In [87]:
df_customer = pd.read_csv("data/customer.csv", sep=';')
df_customer.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


In [205]:
df_customer['deceased_indicator'].unique(), df_customer['owns_car'].unique()

(array(['N', 'Y'], dtype=object), array(['Yes', 'No'], dtype=object))

In [206]:
df_customer['gender'].unique()

array(['F', 'Male', 'Female', 'U', 'Femal', 'M'], dtype=object)

In [207]:
df_customer['gender'].replace({'Female': 'F', 'Femal': 'F', 'Male': 'M'}, inplace=True)

In [208]:
df_customer['country'].unique(), df_customer['state'].unique()

(array(['Australia'], dtype=object),
 array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object))

In [209]:
df_customer['state'].replace({'New South Wales': 'NSW', 'Victoria': 'VIC'}, inplace=True)

In [210]:
df_customer.rename({'DOB': 'dob'}, axis=1, inplace=True)

In [211]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            4000 non-null   int64 
 1   first_name             4000 non-null   object
 2   last_name              3875 non-null   object
 3   gender                 4000 non-null   object
 4   dob                    3913 non-null   object
 5   job_title              3494 non-null   object
 6   job_industry_category  3344 non-null   object
 7   wealth_segment         4000 non-null   object
 8   deceased_indicator     4000 non-null   object
 9   owns_car               4000 non-null   object
 10  address                4000 non-null   object
 11  postcode               4000 non-null   int64 
 12  state                  4000 non-null   object
 13  country                4000 non-null   object
 14  property_valuation     4000 non-null   int64 
dtypes: int64(3), object(1

In [212]:
transaction_names = [
    'transaction_id', 'transaction_date',
    'online_order', 'order_status',
    'list_price', 
    'product_id', 'customer_id',
    'brand',
    'product_line', 'product_class',
    'product_size', 'standard_cost'
    
]

customer_names = [
    'customer_id', 'first_name', 'last_name',
    'gender', 'dob',
    'job_title', 'job_industry_category',
    'wealth_segment', 'property_valuation', 'owns_car',
    'deceased_indicator',
    'address', 'country',
    'state', 'postcode'
]

In [213]:
df_transaction.shape, df_transaction.transaction_id.nunique()

((20000, 12), 20000)

In [214]:
df_customer.shape, df_customer.customer_id.nunique()

((4000, 15), 4000)

In [162]:
# nat_mask = df_customer.dob.apply(lambda x: x is pd.NaT)
# na_mask = df_customer.dob.isna()
# df_customer.loc[nat_mask | na_mask, "dob"] = pd.to_datetime(0) # because postgresql does not accept NaT

In [151]:
%pip install tqdm

  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [215]:
from tqdm import tqdm

for _, row_t in tqdm(df_transaction.iterrows()):
    cursor.execute(f"""
                insert into transaction ({','.join(transaction_names)})
                values (%s{', %s' * (len(transaction_names)-1)})
                on conflict do nothing
                """, row_t[transaction_names])

20000it [00:20, 960.03it/s] 


In [227]:
for _, row_c in tqdm(df_customer.iterrows()):
    cursor.execute(f"""
                insert into customer ({','.join(customer_names)})
                values (%s{', %s' * (len(customer_names)-1)})
                on conflict do nothing
                """, row_c[customer_names])

4000it [00:04, 813.92it/s]


In [228]:
conn.commit()

# Задание 2

### 2.1

In [7]:
conn.rollback()

In [9]:
cursor.execute(f"""
                select job_industry_category, count(customer_id)
                from customer
                group by job_industry_category
                order by count(customer_id) desc
                """)
print(cursor.fetchall())

[('Manufacturing', 799), ('Financial Services', 774), ('NaN', 656), ('Health', 602), ('Retail', 358), ('Property', 267), ('IT', 223), ('Entertainment', 136), ('Argiculture', 113), ('Telecommunications', 72)]


### 2.2

In [13]:
conn.rollback()

In [16]:
cursor.execute(f"""
                select c.job_industry_category, date_part('month', t.transaction_date::timestamp) as month, sum(t.list_price)
                from transaction as t left join customer as c on t.customer_id = c.customer_id
                group by month, c.job_industry_category
                order by month, c.job_industry_category
                """)
print(cursor.fetchall())

[('Argiculture', 1.0, 43513.812), ('Entertainment', 1.0, 64089.934), ('Financial Services', 1.0, 366383.78), ('Health', 1.0, 286860.44), ('IT', 1.0, 107783.414), ('Manufacturing', 1.0, 365232.38), ('NaN', 1.0, 316819.72), ('Property', 1.0, 100686.97), ('Retail', 1.0, 182375.72), ('Telecommunications', 1.0, 31210.2), (None, 1.0, 1102.94), ('Argiculture', 2.0, 60016.81), ('Entertainment', 2.0, 63965.99), ('Financial Services', 2.0, 375961.8), ('Health', 2.0, 268525.75), ('IT', 2.0, 93961.79), ('Manufacturing', 2.0, 389260.53), ('NaN', 2.0, 260711.89), ('Property', 2.0, 112255.41), ('Retail', 2.0, 146107.58), ('Telecommunications', 2.0, 27678.2), ('Argiculture', 3.0, 49048.492), ('Entertainment', 3.0, 77122.44), ('Financial Services', 3.0, 322268.28), ('Health', 3.0, 290575.88), ('IT', 3.0, 100212.4), ('Manufacturing', 3.0, 377784.72), ('NaN', 3.0, 278553.9), ('Property', 3.0, 125143.59), ('Retail', 3.0, 172366.56), ('Telecommunications', 3.0, 24541.277), ('Argiculture', 4.0, 45978.31), (

### 2.3

In [60]:
conn.rollback()

In [56]:
cursor.execute(f"""
                select t.brand, count(t.transaction_id) as counts
                from transaction as t 
                left join customer as c
                on t.customer_id = c.customer_id
                where t.online_order = 'true' and c.job_industry_category = 'IT' 
                group by t.brand
                order by counts desc
                """)
print(cursor.fetchall())

[('Solex', 102), ('Norco Bicycles', 93), ('WeareA2B', 90), ('Giant Bicycles', 89), ('Trek Bicycles', 85), ('OHM Cycles', 78), ('NaN', 8)]


### 2.4

In [112]:
conn.rollback()

In [ ]:
cursor.execute(f"""
                select customer_id, sum(list_price) as total_sum, 
                max(list_price), min(list_price), count(*) as total_count
                from transaction
                group by customer_id
                order by total_sum desc, total_count desc
                """)
result_group_by = cursor.fetchall()
result_group_by

In [144]:
conn.rollback()

In [160]:
cursor.execute(f"""
                select customer_id, sum(list_price::float8) as total_sum, 
                max(list_price), min(list_price), count(*) as total_count
                from transaction
                group by customer_id
                order by total_sum desc, total_count desc
                """)
result_group_by = cursor.fetchall()
result_group_by

[(2183, 19071.320251464844, 2005.66, 230.91, 14),
 (1129, 18349.270141601562, 1992.93, 290.62, 13),
 (1597, 18052.67984008789, 2091.47, 360.4, 12),
 (941, 17898.4599609375, 2091.47, 1057.51, 10),
 (2788, 17258.939895629883, 2083.94, 183.86, 11),
 (936, 17160.239944458008, 2005.66, 183.86, 12),
 (1887, 17133.93017578125, 2091.47, 688.63, 11),
 (1302, 17035.82992553711, 1977.36, 71.16, 13),
 (1140, 16199.239944458008, 2083.94, 183.86, 13),
 (2309, 16122.340118408203, 2091.47, 290.62, 12),
 (729, 15825.999938964844, 2091.47, 586.45, 10),
 (1103, 15447.92007446289, 1977.36, 230.91, 12),
 (1317, 15370.809997558594, 2091.47, 569.56, 9),
 (2874, 15091.910278320312, 2005.66, 544.05, 11),
 (2762, 15071.260147094727, 1997.68, 183.86, 10),
 (255, 14949.910034179688, 1810.0, 441.49, 12),
 (1672, 14908.46011352539, 1977.36, 363.01, 13),
 (1329, 14818.119873046875, 1793.43, 543.39, 11),
 (322, 14782.23009300232, 2005.66, 12.01, 12),
 (3048, 14756.089965820312, 1890.39, 543.39, 13),
 (2860, 14673.349

In [166]:
cursor.execute(f"""
               select customer_id, total_sum, total_max, total_min, total_count
               from (
                select customer_id, 
                sum(list_price::float8) over ( partition by customer_id ) as total_sum, 
                max(list_price) over ( partition by customer_id ) as total_max, 
                min(list_price) over ( partition by customer_id ) as total_min, 
                count(*) over ( partition by customer_id ) as total_count,
                row_number() over ( partition by customer_id ) as row_num
                from transaction
                )
                where row_num = 1
                order by total_sum desc, total_count desc
                """)
result_window = cursor.fetchall()
result_window

[(2183, 19071.320251464844, 2005.66, 230.91, 14),
 (1129, 18349.270141601562, 1992.93, 290.62, 13),
 (1597, 18052.67984008789, 2091.47, 360.4, 12),
 (941, 17898.4599609375, 2091.47, 1057.51, 10),
 (2788, 17258.939895629883, 2083.94, 183.86, 11),
 (936, 17160.239944458008, 2005.66, 183.86, 12),
 (1887, 17133.93017578125, 2091.47, 688.63, 11),
 (1302, 17035.82992553711, 1977.36, 71.16, 13),
 (1140, 16199.239944458008, 2083.94, 183.86, 13),
 (2309, 16122.340118408203, 2091.47, 290.62, 12),
 (729, 15825.999938964844, 2091.47, 586.45, 10),
 (1103, 15447.92007446289, 1977.36, 230.91, 12),
 (1317, 15370.809997558594, 2091.47, 569.56, 9),
 (2874, 15091.910278320312, 2005.66, 544.05, 11),
 (2762, 15071.260147094727, 1997.68, 183.86, 10),
 (255, 14949.910034179688, 1810.0, 441.49, 12),
 (1672, 14908.46011352539, 1977.36, 363.01, 13),
 (1329, 14818.119873046875, 1793.43, 543.39, 11),
 (322, 14782.23009300232, 2005.66, 12.01, 12),
 (3048, 14756.089965820312, 1890.39, 543.39, 13),
 (2860, 14673.349

In [72]:
len(result_group_by), len(result_window)

(3494, 3494)

In [85]:
cursor.execute(f"""
               select customer_id, list_price, count(*) as dcounts
               from transaction
               group by customer_id, list_price
               having count(*) > 1
                """)
print(cursor.fetchall())

[(1993, 1403.5, 3), (394, 1483.2, 2), (3280, 1762.96, 2), (1560, 60.34, 2), (1648, 1403.5, 2), (2183, 2005.66, 2), (621, 1992.93, 2), (396, 1231.15, 2), (1274, 235.63, 2), (2169, 1945.43, 2), (594, 1769.64, 2), (2465, 1458.17, 2), (3176, 742.54, 2), (3335, 1057.51, 2), (1150, 1890.39, 2), (2476, 1240.31, 2), (809, 742.54, 2), (1627, 1228.07, 2), (1101, 642.7, 2), (2069, 1992.93, 2), (2884, 175.89, 2), (3476, 230.91, 2), (3439, 792.9, 2), (280, 1765.3, 2), (611, 1775.81, 2), (1866, 416.98, 2), (999, 2091.47, 2), (1282, 688.63, 2), (545, 2091.47, 2), (739, 774.53, 2), (1912, 12.01, 2), (2686, 1793.43, 2), (2750, 1228.07, 2), (2046, 2091.47, 2), (8, 1129.13, 2), (3158, 1148.64, 2), (2637, 2091.47, 2), (1625, 2091.47, 3), (969, 1240.31, 2), (2070, 2091.47, 2), (346, 912.52, 2), (2162, 792.9, 2), (75, 1057.51, 2), (186, 1274.93, 2), (1558, 2091.47, 2), (655, 2091.47, 2), (1999, 1240.31, 2), (2697, 1636.9, 2), (3154, 1311.44, 2), (1015, 1403.5, 2), (3133, 1810.0, 2), (2700, 1073.07, 2), (288

In [90]:
df_transaction.list_price = df_transaction.list_price.apply(lambda x: float(str(x).replace(',', '.')))

In [156]:
df = df_transaction.groupby('customer_id', as_index=False).list_price.sum()
df[df.customer_id == 1129]

,customer_id,list_price
1126,1129,18349.27


Единственное различие между методами кроется в возвращаемой сумме. В данных есть дубликаты по customer_id и list_price, поэтому group by аггрегирует дубликаты в одну строку и получит неверный результат 

### 2.5

### 2.6

### 2.7

# Завершение работы

In [272]:
cursor.close()
conn.close()